# Narupa ASE with Interactive Molecular Dynamics

This notebook runs an OpenMM simulation of a nanotube with ASE, then uses the client to apply a force to it and NGLViewer to visualise it

In [1]:
from narupa.app.client import NarupaImdClient
from narupa.ase.openmm import OpenMMIMDRunner
from narupa.ase.converter import frame_data_to_ase
from narupa.imd.particle_interaction import ParticleInteraction
import numpy as np
import nglview
import MDAnalysis as mda
import matplotlib.pyplot as plt
from queue import Queue

_ColormakerRegistry()

## Run the server

Run the server. Here, we're using ASE, but it could be LAMMPS or OpenMM or anything else. 

In [2]:
# set up an openmm simulation, or read from file. 
input_xml = "nanotube.xml"


In [3]:
runner = OpenMMIMDRunner.from_xml(input_xml)

C:\Users\Mark\Anaconda3\envs\narupa-dev\lib\site-packages\simtk\openmm\app\internal\pdbstructure.py:536: UserWarning: WARNING: duplicate atom (HETATM    2 mm3_ mm3 A   1      30.047  35.164  32.988  1.00  0.00           C  , HETATM    1 mm3_ mm3 A   1      29.729  34.950  34.124  1.00  0.00           C  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
C:\Users\Mark\Anaconda3\envs\narupa-dev\lib\site-packages\simtk\openmm\app\internal\pdbstructure.py:536: UserWarning: WARNING: duplicate atom (HETATM    3 mm3_ mm3 A   1      31.361  35.154  32.407  1.00  0.00           C  , HETATM    2 mm3_ mm3 A   1      30.047  35.164  32.988  1.00  0.00           C  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
C:\Users\Mark\Anaconda3\envs\narupa-dev\lib\site-packages\simtk\openmm\app\internal\pdbstructure.py:53

Run dynamics in background thread

In [4]:
runner.run()

In [5]:
# print the time to check dynamics is running
runner.dynamics.get_time()

0.7858155830771251

## Start an IMD client

In [6]:
class NGLClient(NarupaImdClient):
    def __init__(self, *args, update_callback=None, **kwargs):
        super().__init__(*args, **kwargs)
        self._view = None
        self.update_callback = update_callback
    
    @property
    def view(self):
        if self._view is None:
            atoms = frame_data_to_ase(
                client.first_frame,
                topology=True,
                positions=False,
            )
            atoms.set_positions(
                np.array(client.latest_frame.particle_positions) * 10
            )
            self._view = nglview.show_ase(atoms)
        return self._view

    def _on_frame_received(self, frame_index:int, frame):
        super()._on_frame_received(frame_index, frame)
        self.view.set_coordinates(
            {0: np.array(self.latest_frame.particle_positions) * 10}
        )
        if self.update_callback is not None:
            self.update_callback(self.universe)

            
def _get_structure_string(write_method, suffix='.pdb'):
    fname = 'tmp.pdb'
    write_method(fname)
    with open(fname) as fh:
        return fh.read()

nglview.adaptor._get_structure_string = _get_structure_string

In [7]:
client = NGLClient()

In [8]:
positions = np.array(client.latest_frame.particle_positions) * 10
start_point = positions[60, :]
first_nanotube_anchor = positions[0, :]
last_nanotube_anchor = positions[59, :]

In [9]:
anchor1 = ParticleInteraction(position=first_nanotube_anchor, scale=5000, particles=(0,), interaction_type='gaussian')
anchor2 = ParticleInteraction(position=last_nanotube_anchor, scale=5000, particles=(59,), interaction_type='gaussian')

anchor1_id = client.start_interaction()
client.update_interaction(anchor1_id, anchor1)
anchor2_id = client.start_interaction()
client.update_interaction(anchor2_id, anchor2)

In [10]:
interaction = ParticleInteraction(position=start_point, scale=10000, particles=(60,), interaction_type='spring')
interaction_id = client.start_interaction()
client.update_interaction(interaction_id, interaction)

In [11]:
client.view

NGLWidget()

In [12]:
client.view.shape.add_sphere(list(start_point), [1, 0, 0], 0.5)
client.view.shape.add_sphere(list(first_nanotube_anchor), [1, 0, 1], 0.5)
client.view.shape.add_sphere(list(last_nanotube_anchor), [1, 1, 0], 0.5)

In [13]:
client.first_frame.raw

values {
  key: "chain.count"
  value {
    number_value: 1.0
  }
}
values {
  key: "energy.kinetic"
  value {
    number_value: 2051.4897728419737
  }
}
values {
  key: "energy.potential"
  value {
    number_value: 2300.906494140625
  }
}
values {
  key: "particle.count"
  value {
    number_value: 65.0
  }
}
values {
  key: "residue.count"
  value {
    number_value: 1.0
  }
}
arrays {
  key: "bond.pairs"
  value {
    index_values {
      values: 0
      values: 1
      values: 0
      values: 11
      values: 1
      values: 2
      values: 2
      values: 3
      values: 2
      values: 13
      values: 3
      values: 4
      values: 3
      values: 16
      values: 4
      values: 5
      values: 5
      values: 6
      values: 6
      values: 7
      values: 6
      values: 17
      values: 7
      values: 8
      values: 7
      values: 20
      values: 8
      values: 9
      values: 9
      values: 10
      values: 10
      values: 11
      values: 10
      values: 21
     